In [19]:
import requests
import xml.etree.ElementTree as ET

auth_key = "c806e29e6e85e4ed20d0563d93ebf0bd1f272962598876ea14c651141d8bb18d"
url = f"http://data4library.kr/api/loanItemSrch?authKey={auth_key}&pageSize=10"

# API 호출 및 XML 데이터 파싱
response = requests.get(url)
response.raise_for_status()
root = ET.fromstring(response.content)

# 'docs' 요소를 찾고, 그 하위의 'doc' 요소에서 ISBN 번호 추출
docs = root.find("docs")
if docs is not None:
    isbn_list = [doc.find("isbn13").text.strip() for doc in docs.findall("doc") if doc.find("isbn13") is not None]
else:
    isbn_list = []

print(isbn_list)

['9788972756194', '9788936433673', '9788956604992', '9788973374113', '9788976491114', '9788936433598', '9788996991342', '9788936434267', '9788990611260', '9788937473135']


In [20]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import tqdm

# API 키를 문자열로 정의
auth_key = "c806e29e6e85e4ed20d0563d93ebf0bd1f272962598876ea14c651141d8bb18d"



# 결과 저장을 위한 빈 리스트
book_info_data = []
loan_info_data = []
loan_grp_data = []
keywords_data = []
co_loan_books_data = []
mania_rec_books_data = []
reader_rec_books_data = []

# 각 ISBN에 대해 API 호출 및 데이터 수집
for isbn in tqdm.tqdm(isbn_list, desc="Processing ISBNs"):
    url = f"http://data4library.kr/api/usageAnalysisList?authKey={auth_key}&isbn13={isbn}"
    
    try:
        response = requests.get(url)
        response.raise_for_status()  # HTTP 오류 발생 시 예외를 발생시킵니다.

        # XML 응답 파싱
        root = ET.fromstring(response.text)

        # 도서 서지 정보 데이터 추출
        for book in root.findall(".//book"):
            try:
                bookname = book.find("bookname").text if book.find("bookname") is not None else "N/A"
                authors = book.find("authors").text if book.find("authors") is not None else "N/A"
                publisher = book.find("publisher").text if book.find("publisher") is not None else "N/A"
                book_image_url = book.find("bookImageURL").text if book.find("bookImageURL") is not None else "N/A"
                description = book.find("description").text if book.find("description") is not None else "N/A"
                publication_year = book.find("publication_year").text if book.find("publication_year") is not None else "N/A"
                isbn13 = book.find("isbn13").text if book.find("isbn13") is not None else "N/A"
                vol = book.find("vol").text if book.find("vol") is not None else "N/A"
                class_no = book.find("class_no").text if book.find("class_no") is not None else "N/A"
                class_nm = book.find("class_nm").text if book.find("class_nm") is not None else "N/A"
                loan_cnt = int(book.find("loanCnt").text) if book.find("loanCnt") is not None else 0
                book_info_data.append({"도서명": bookname, "저자명": authors, "출판사": publisher,
                                       "책표지 URL": book_image_url, "책소개": description, "출판년도": publication_year,
                                       "13 자리 ISBN": isbn13, "권": vol, "주제분류": class_no, "주제분류명": class_nm, "대출건수": loan_cnt})
            except Exception as e:
                print(f"Error processing book data: {e}")

        # 대출 정보 데이터 추출
        for loan in root.findall(".//loan"):
            try:
                month = loan.find("month").text if loan.find("month") is not None else "N/A"
                loan_cnt = int(loan.find("loanCnt").text) if loan.find("loanCnt") is not None else 0
                ranking = int(loan.find("ranking").text) if loan.find("ranking") is not None else 0
                loan_info_data.append({"대출년월": month, "대출건수": loan_cnt, "순위": ranking})
            except Exception as e:
                print(f"Error processing loan data: {e}")

        # 다대출 이용자 그룹 데이터 추출
        for grp in root.findall(".//loanGrps"):
            try:
                age = grp.find("age").text if grp.find("age") is not None else "N/A"
                gender = grp.find("gender").text if grp.find("gender") is not None else "N/A"
                loan_cnt = int(grp.find("loanCnt").text) if grp.find("loanCnt") is not None else 0
                ranking = int(grp.find("ranking").text) if grp.find("ranking") is not None else 0
                loan_grp_data.append({"연령": age, "성별": gender, "대출건수": loan_cnt, "순위": ranking})
            except Exception as e:
                print(f"Error processing loan group data: {e}")

        # 키워드 정보 데이터 추출
        for keyword in root.findall(".//keywords/keyword"):
            try:
                word = keyword.find("word").text if keyword.find("word") is not None else "N/A"
                weight = float(keyword.find("weight").text) if keyword.find("weight") is not None else 0.0
                keywords_data.append({"키워드": word, "가중치": weight})
            except Exception as e:
                print(f"Error processing keyword data: {e}")

        # 함께 대출된 도서 데이터 추출
        for book in root.findall(".//coLoanBooks/book"):
            try:
                bookname = book.find("bookname").text if book.find("bookname") is not None else "N/A"
                authors = book.find("authors").text if book.find("authors") is not None else "N/A"
                publisher = book.find("publisher").text if book.find("publisher") is not None else "N/A"
                publication_year = book.find("publication_year").text if book.find("publication_year") is not None else "N/A"
                isbn13 = book.find("isbn13").text if book.find("isbn13") is not None else "N/A"
                vol = book.find("vol").text if book.find("vol") is not None else "N/A"
                loan_cnt = int(book.find("loanCnt").text) if book.find("loanCnt") is not None else 0
                co_loan_books_data.append({"도서명": bookname, "저자명": authors, "출판사": publisher,
                                           "출판년도": publication_year, "13 자리 ISBN": isbn13, "권": vol, "대출건수": loan_cnt})
            except Exception as e:
                print(f"Error processing co-loan books data: {e}")

        # 마니아 추천 도서 데이터 추출
        for book in root.findall(".//maniaRecBooks/book"):
            try:
                bookname = book.find("bookname").text if book.find("bookname") is not None else "N/A"
                authors = book.find("authors").text if book.find("authors") is not None else "N/A"
                publisher = book.find("publisher").text if book.find("publisher") is not None else "N/A"
                publication_year = book.find("publication_year").text if book.find("publication_year") is not None else "N/A"
                isbn13 = book.find("isbn13").text if book.find("isbn13") is not None else "N/A"
                vol = book.find("vol").text if book.find("vol") is not None else "N/A"
                mania_rec_books_data.append({"도서명": bookname, "저자명": authors, "출판사": publisher,
                                             "출판년도": publication_year, "13 자리 ISBN": isbn13, "권": vol})
            except Exception as e:
                print(f"Error processing mania rec books data: {e}")

        # 다독자 추천 도서 데이터 추출
        for book in root.findall(".//readerRecBooks/book"):
            try:
                bookname = book.find("bookname").text if book.find("bookname") is not None else "N/A"
                authors = book.find("authors").text if book.find("authors") is not None else "N/A"
                publisher = book.find("publisher").text if book.find("publisher") is not None else "N/A"
                publication_year = book.find("publication_year").text if book.find("publication_year") is not None else "N/A"
                isbn13 = book.find("isbn13").text if book.find("isbn13") is not None else "N/A"
                vol = book.find("vol").text if book.find("vol") is not None else "N/A"
                reader_rec_books_data.append({"도서명": bookname, "저자명": authors, "출판사": publisher,
                                               "출판년도": publication_year, "13 자리 ISBN": isbn13, "권": vol})
            except Exception as e:
                print(f"Error processing reader rec books data: {e}")

    except requests.RequestException as e:
        print(f"Request failed: {e}")

# 데이터 프레임 생성
df_book_info = pd.DataFrame(book_info_data)
df_loan_info = pd.DataFrame(loan_info_data)
df_loan_grp = pd.DataFrame(loan_grp_data)
df_keywords = pd.DataFrame(keywords_data)
df_co_loan_books = pd.DataFrame(co_loan_books_data)
df_mania_rec_books = pd.DataFrame(mania_rec_books_data)
df_reader_rec_books = pd.DataFrame(reader_rec_books_data)

# Excel 파일로 저장
with pd.ExcelWriter("book_analysis.xlsx") as writer:
    df_book_info.to_excel(writer, sheet_name="도서 서지 정보", index=False)
    df_loan_info.to_excel(writer, sheet_name="대출 정보", index=False)
    df_loan_grp.to_excel(writer, sheet_name="다대출 이용자 그룹", index=False)
    df_keywords.to_excel(writer, sheet_name="키워드 정보", index=False)
    df_co_loan_books.to_excel(writer, sheet_name="함께 대출된 도서", index=False)
    df_mania_rec_books.to_excel(writer, sheet_name="마니아 추천 도서", index=False)
    df_reader_rec_books.to_excel(writer, sheet_name="다독자 추천 도서", index=False)

print("Data has been successfully saved to book_analysis.xlsx")


Processing ISBNs: 100%|█████████████████████████| 10/10 [00:05<00:00,  1.85it/s]


Data has been successfully saved to book_analysis.xlsx
